In [1]:
import geopandas as gp
import pandas as pd
import os
import math
import pber_functions_v1 as pdv
from functools import reduce

# Louisiana 2024 General Precinct-Level Election Results

## Load in Raw Files

In [2]:
#Load in clean SOS file
elections_2024_raw = pd.read_excel("./raw-from-source/Election+Results+(11-05-2024).xlsx",sheet_name="Multi-Parish(Precinct)")

# Drop the first 4 rows
elections_2024_raw = elections_2024_raw[4:]

# Reset index
elections_2024_raw.reset_index(inplace = True, drop = True)

# Clean "Unnamed: 0" column
elections_2024_raw["Unnamed: 0"] = elections_2024_raw["Unnamed: 0"].fillna("NONE")

# Take a look at the file
display(elections_2024_raw.head())


/Users/peterhorton/opt/anaconda3/envs/run_maup/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Unnamed: 0,Louisiana Secretary of State,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Presidential Electors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NONE,"Kamala D. Harris, Tim Walz Democratic (DEM)","Jill Stein, Ajamu Baraka Green (GRN)","Chase Oliver, Mike ter Maat Libertarian (LBT)","Donald J. Trump, ""JD"" Vance Republican (REP)","Peter Sonski, Lauren Onak American Solidarity ...","Randall Terry, Stephen Broden Constitution Par...","Mattie Preston, Shannel Conner Godliness, Trut...","Cornel West, Melina Abdullah Justice For All (...","Claudia de la Cruz, Karina Garcia Socialism an...","Rachele Fruit, Dennis Richter Socialist Worker...","Robert F. Kennedy, Jr., Nicole Shanahan We The..."
2,Total Votes,766870,7138,6835,1208505,2240,1424,2857,2623,1481,361,6641
3,Acadia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01 01,58,0,5,624,3,0,0,0,0,0,3


In [3]:
slice_list = []
prev = 0
for val in range(1, len(elections_2024_raw)):
    if elections_2024_raw.loc[val, "Unnamed: 0"]=="NONE" and elections_2024_raw.loc[val+1, "Unnamed: 0"]=="NONE" :
        print(str(val))
        print(str(val+1))
        print(slice(val,val+1))
        #slice_list.append(slice(val,val+1))
        slice_list.append(slice(prev,val-1))
        prev = val+2
slice_list.append(slice(prev, len(elections_2024_raw)-1 ))

4000
4001
slice(4000, 4001, None)
4563
4564
slice(4563, 4564, None)
5215
5216
slice(5215, 5216, None)
5810
5811
slice(5810, 5811, None)
6644
6645
slice(6644, 6645, None)
7397
7398
slice(7397, 7398, None)
8082
8083
slice(8082, 8083, None)
8373
8374
slice(8373, 8374, None)
8604
8605
slice(8604, 8605, None)
9302
9303
slice(9302, 9303, None)
9478
9479
slice(9478, 9479, None)
9594
9595
slice(9594, 9595, None)
9606
9607
slice(9606, 9607, None)
9618
9619
slice(9618, 9619, None)
9629
9630
slice(9629, 9630, None)


In [4]:
if not os.path.exists("./cleaned_results_general_unallocated/"):
    os.mkdir("./cleaned_results_general_unallocated/")

In [5]:
for slice_val in slice_list:
    subset = elections_2024_raw.loc[slice_val]
    subset.reset_index(inplace = True, drop = True)
    contest = subset.loc[0,"Unnamed: 0"]
    
    #if True:
    print(contest)
    if True:
        #print(subset.loc[1])
        subset.columns = [str(i)+ "-:-" + contest for i in subset.loc[1] ]
        subset = subset.loc[2:]
        subset.reset_index(inplace = True, drop = True)
        #print(subset)
        
        county = "Statewide"
        county_dict = {}
        for val in range(0, len(subset[subset.columns[1]])):
            if math.isnan(subset.loc[val, subset.columns[1]]):
                #print(val)
                county = subset.loc[val,subset.columns[0]]
            county_dict[val] = county
        subset["COUNTY"] = subset.index.map(county_dict).fillna("BREAKPOINT")
        assert(subset[subset["COUNTY"]=="BREAKPOINT"].shape[0]==0)
        subset.rename(columns = {subset.columns[0]:"Precinct"}, inplace = True)        
        subset = subset[~subset[subset.columns[1]].isna()]
        
        subset.drop([i for i in subset.columns if i[:3] == "nan"], axis = 1, inplace = True)
        
        first_column = subset.pop('COUNTY')
        subset.insert(0, 'COUNTY', first_column)
        
        # Allocate results
        subset["UNIQUE_ID"] = subset["COUNTY"]+"-:-"+subset["Precinct"]
        race_cols = [i for i in subset.columns if i not in ["UNIQUE_ID","COUNTY","Precinct"]]
        #subset["PARISH"] = subset["UNIQUE_ID"].apply(lambda x: x.split("-:-")[0])
        statewide_vals = subset[subset["COUNTY"]=="Statewide"]
        print(statewide_vals.iloc[0])
        subset = subset[subset["COUNTY"]!="Statewide"]
#         allocate_precs = subset[subset["UNIQUE_ID"].str.contains("Provisional") | subset["UNIQUE_ID"].str.contains("Early Voting")]
#         receiving_votes = subset[~(subset["UNIQUE_ID"].str.contains("Provisional") | subset["UNIQUE_ID"].str.contains("Early Voting"))]
        
#         subset_allocated = pdv.allocate_absentee(receiving_votes, allocate_precs, race_cols, "COUNTY", allocating_to_all_empty_precs=False)
        
        
        subset.to_csv("./cleaned_results_general_unallocated/" + contest + ".csv", index = False)    

Presidential Electors
COUNTY                                                                                                      Statewide
Precinct                                                                                                  Total Votes
Kamala D. Harris, Tim Walz Democratic (DEM)-:-Presidential Electors                                            766870
Jill Stein, Ajamu Baraka Green (GRN)-:-Presidential Electors                                                     7138
Chase Oliver, Mike ter Maat Libertarian (LBT)-:-Presidential Electors                                            6835
Donald J. Trump, "JD" Vance Republican (REP)-:-Presidential Electors                                          1208505
Peter Sonski, Lauren Onak American Solidarity Party (OTHER)-:-Presidential Electors                              2240
Randall Terry, Stephen Broden Constitution Party (OTHER)-:-Presidential Electors                                 1424
Mattie Preston, Shannel Conner God

Mayor -- Town of Delcambre
COUNTY                                                                           Statewide
Precinct                                                                       Total Votes
Pam Blakely (IND)-:-Mayor -- Town of Delcambre                                         360
Christopher "Tipper" Esponge (REP)-:-Mayor -- Town of Delcambre                        428
UNIQUE_ID                                                          Statewide-:-Total Votes
Name: 0, dtype: object
Chief of Police -- Town of Delcambre
COUNTY                                                                      Statewide
Precinct                                                                  Total Votes
Ivan McIntyre (REP)-:-Chief of Police -- Town of Delcambre                        129
Brady Segura (REP)-:-Chief of Police -- Town of Delcambre                         241
Perry Shaw (IND)-:-Chief of Police -- Town of Delcambre                           416
UNIQUE_ID                   

## Reload the Election Results File, Process So Each Row is a Precinct

In [6]:
path = "./cleaned_results_general_unallocated/"
holder = []
for file in os.listdir(path):
    if file != ".DS_Store":
        temp = pd.read_csv(path+file)
        print(temp.columns)
        print(file)
        print("")
        if file != "Presidential Electors.csv":
            temp.drop(["COUNTY", "Precinct"], axis = 1, inplace = True)
        else:
            temp.rename(columns = {"COUNTY":"PARISH"}, inplace = True)
        holder.append(temp)

Index(['COUNTY', 'Precinct',
       'YES-:-CA No. 1 (Act No. 408, 2024 - HB 300) -- Provides relative to federal proceeds from alternative energy production',
       'NO-:-CA No. 1 (Act No. 408, 2024 - HB 300) -- Provides relative to federal proceeds from alternative energy production',
       'UNIQUE_ID'],
      dtype='object')
CA No. 1 (Act No. 408, 2024 - HB 300) -- Provides relative to federal proceeds from alternative energy production.csv

Index(['COUNTY', 'Precinct',
       'Kelly Balfour (REP)-:-Judge, Court of Appeal -- 1st Circuit, 2nd Dist., Subdist. 1, Div. C',
       'Eboni Johnson Rose (DEM)-:-Judge, Court of Appeal -- 1st Circuit, 2nd Dist., Subdist. 1, Div. C',
       'UNIQUE_ID'],
      dtype='object')
Judge, Court of Appeal -- 1st Circuit, 2nd Dist., Subdist. 1, Div. C.csv

Index(['COUNTY', 'Precinct', 'Pam Blakely (IND)-:-Mayor -- Town of Delcambre',
       'Christopher "Tipper" Esponge (REP)-:-Mayor -- Town of Delcambre',
       'UNIQUE_ID'],
      dtype='object')
M

In [7]:
la_24_general = reduce(lambda x, y: pd.merge(x, y, on = "UNIQUE_ID", how = "outer"), holder)
first_column = la_24_general.pop('UNIQUE_ID')
la_24_general.insert(0, 'UNIQUE_ID', first_column)
la_24_general = la_24_general.fillna(0)

In [8]:
other_ones = [i for i in list(la_24_general.columns) if i in ["UNIQUE_ID","PARISH","Precinct"] or "SCC" in i or "Presidential" in i or "Congressional" in i or "No." in i]
la_24_general = la_24_general[other_ones]

In [9]:
la_24_general_rename_dict = {i:i.replace("SCC Member --   ","SCC Member -- ") for i in list(la_24_general.columns)}

In [10]:
la_24_general.rename(columns = la_24_general_rename_dict, inplace = True)

In [11]:
la_24_general

,UNIQUE_ID,"YES-:-CA No. 1 (Act No. 408, 2024 - HB 300) -- Provides relative to federal proceeds from alternative energy production","NO-:-CA No. 1 (Act No. 408, 2024 - HB 300) -- Provides relative to federal proceeds from alternative energy production",PARISH,Precinct,"Kamala D. Harris, Tim Walz Democratic (DEM)-:-Presidential Electors","Jill Stein, Ajamu Baraka Green (GRN)-:-Presidential Electors","Chase Oliver, Mike ter Maat Libertarian (LBT)-:-Presidential Electors","Donald J. Trump, ""JD"" Vance Republican (REP)-:-Presidential Electors","Peter Sonski, Lauren Onak American Solidarity Party (OTHER)-:-Presidential Electors",...,"J. ""Frankie"" Hyers (NOPTY)-:-U. S. Representative -- 1st Congressional District",Mel Manuel (DEM)-:-U. S. Representative -- 1st Congressional District,Steve Scalise (REP)-:-U. S. Representative -- 1st Congressional District,Ross Shales (REP)-:-U. S. Representative -- 1st Congressional District,"""Mike"" Johnson (REP)-:-U. S. Representative -- 4th Congressional District",Joshua Morott (REP)-:-U. S. Representative -- 4th Congressional District,Priscilla Gonzalez (DEM)-:-U. S. Representative -- 3rd Congressional District,Clay Higgins (REP)-:-U. S. Representative -- 3rd Congressional District,"""Xan"" John (REP)-:-U. S. Representative -- 3rd Congressional District",Sadi Summerlin (DEM)-:-U. S. Representative -- 3rd Congressional District
0,Acadia-:-01 01,479.0,154.0,Acadia,01 01,58,0,5,624,3,...,0.0,0.0,0.0,0.0,0.0,0.0,49.0,593.0,20.0,12.0
1,Acadia-:-01 02,286.0,142.0,Acadia,01 02,114,1,2,352,1,...,0.0,0.0,0.0,0.0,0.0,0.0,74.0,335.0,16.0,23.0
2,Acadia-:-01 03,316.0,145.0,Acadia,01 03,94,2,1,408,1,...,0.0,0.0,0.0,0.0,0.0,0.0,69.0,395.0,6.0,11.0
3,Acadia-:-01 04,371.0,163.0,Acadia,01 04,99,3,2,493,1,...,0.0,0.0,0.0,0.0,0.0,0.0,73.0,454.0,25.0,17.0
4,Acadia-:-01 05,110.0,80.0,Acadia,01 05,123,2,1,101,0,...,0.0,0.0,0.0,0.0,0.0,0.0,81.0,102.0,0.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3928,Winn-:-07 2,45.0,58.0,Winn,07 2,1,0,0,108,0,...,0.0,0.0,0.0,0.0,92.0,4.0,0.0,0.0,0.0,0.0
3929,Winn-:-07 2A,68.0,56.0,Winn,07 2A,17,0,0,122,0,...,0.0,0.0,0.0,0.0,112.0,10.0,0.0,0.0,0.0,0.0
3930,Winn-:-07 7,52.0,49.0,Winn,07 7,7,0,0,104,0,...,0.0,0.0,0.0,0.0,93.0,10.0,0.0,0.0,0.0,0.0
3931,Winn-:-Early Voting,1667.0,984.0,Winn,Early Voting,489,1,4,2232,1,...,0.0,0.0,0.0,0.0,2325.0,295.0,0.0,0.0,0.0,0.0


In [12]:
def get_race(race_string):
    race_string = race_string.title()
    race_string = race_string.replace("(Vote For 1)","")
    if "U.S. House" in race_string or 'Us House' in race_string or "Representative To Congress" in race_string or "U. S. Representative" in race_string:
        return "CON"
    elif "State House" in race_string or "State Representative" in race_string:
        return "SL"
    elif "State Senate" in race_string or "State Senator" in race_string:
        return "SU"
    elif "President" in race_string:
        return "PRE"
    elif "US Senate" in race_string or "Us Senate" in race_string or "U.S. Senator" in race_string or "U. S. Senator" in race_string:
        return "USS"
    elif "Public Service" in race_string:
        return "PSC"
    elif "Attorney General" in race_string:
        return "ATG"
    elif "Auditor General" in race_string or "Auditor" in race_string:
        return "AUD"
    elif "Treasurer" in race_string:
        return "TRE"
    elif "Superintendent" in race_string:
        return "SUP"
    elif "Secretary Of State" in race_string:
        return "SOS"
    elif "Lieutenant Governor" in race_string:
        return "LTG"
    elif "Governor" in race_string:
        return "GOV"
    elif "Commissioner Of Labor" in race_string:
        return "LAB"
    elif "Commissioner Of Agriculture" in race_string:
        return "AGR"
    elif "Commissioner Of Insurance" in race_string:
        return "INS"
    elif "Chief Justice Of The Supreme Court" in race_string:
        return "CJU"
    elif "Justice Of The Supreme Court" in race_string:
        return "JUS"
    elif "Ca No." in race_string:
#         num = race_string.split(" ")[4][:-1]
        return "A1"
    elif "Rscc" in race_string or "Dscc" in race_string:
        return "SC"

    else:
        print("No race for:", race_string)
        raise ValueError
        
def get_election_type(race_string):
    if "CA No." in race_string:
        return ""
    else:
        return "R"
        
def get_party(race_string):
    if "(REP)" in race_string:
        return "R"
    elif "(DEM)" in race_string or "(Democrat)" in race_string:
        return "D"
    elif "(L)" in race_string or "(LBT)" in race_string:
        return "L"
    elif "(WI)" in race_string:
        return "O"
    elif "(IND)" in race_string:
        return "I"
    elif "(NOPTY)" in race_string:
        return "N"
    elif race_string[0:3]=="YES":
        return "YES"
    elif race_string[0:2]=="NO":
        return "NO"
    else:
        return "O"
           
def get_name(name_string):
    if "No." in name_string:
        return ""
    elif "Presidential" in name_string:
        name_string = name_string.split(", ")[0]
        name_string = name_string.replace("'","")
        likely_last = name_string.split(" ")[-1]
        proposed_last = likely_last[:3]
        if proposed_last in ['II', 'III', 'Jr', 'Jr.', 'Sr.', 'JR.', "JR", "IV", 'I']:
            likely_last = name_string.split(" ")[-2]
            proposed_last = likely_last[:3]
        #print(proposed_last.upper())
        return proposed_last.upper()        
    elif " (" in name_string and "Amendment" not in name_string and "Referendum" not in name_string:
        #print(name_string)
        name_string = name_string.split(" (")[0]
        name_string = name_string.replace("'","")
        likely_last = name_string.split(" ")[-1]
        proposed_last = likely_last[:3]
        if proposed_last in ['II', 'III', 'Jr', 'Jr.', 'Sr.', 'JR.', "JR", "IV", 'I']:
            likely_last = name_string.split(" ")[-2]
            proposed_last = likely_last[:3]
        #print(proposed_last.upper())
        return proposed_last.upper()
    else:
        return name_string.split(" ")[-1][:3].upper()
import re
 

def get_district(race_string, fill_level):
    temp = race_string.split(" -- ")[1].strip().split(" ")[0]
    return re.sub("\D","",temp).zfill(fill_level)

def get_district_advanced(race_string):
    if " --  " in race_string:
        split = " --  "
    else:
        split = " -- "
    if "Representative" in race_string:
        half = race_string.split(split)[1]
        num = half.split(" ")[0][:-2]
        char = half.split(" ")[-1][1]
        return (num+char).zfill(4)
    else:
        return race_string.split(" ")[-1].zfill(4)
    

def column_rename_function(name_string):
    election_type = "G"
    year = "24"
    party = get_party(name_string)
    race = get_race(name_string)
    district = ""
    #print(name_string)
    if race in ["CON", "SU"]:
        district = get_district(name_string, 2)
        year = ""
    elif race in ["SL"]:
        district = get_district(name_string, 3)
        year = ""
    elif race in ["SC"]:
        year = ""
        district = get_district_advanced(name_string)
        election_type = ""
    name = get_name(name_string)
    new_col_name = election_type + year + race + district + party + name
#     print("election_type: ", election_type)
#     print("year: ", year)
#     print("race: ", race)
#     print("district: ", district)
#     print("party: ", party)
#     print("name: ", name)
    if len(new_col_name) > 10:
        print("LONG NAME")
        print(name_string, "->", new_col_name)
    return new_col_name

races_to_clean = [i for i in la_24_general.columns if i not in ["UNIQUE_ID", "PARISH", "Precinct"]]


race_updates_dict = {}
race_updates_reversed = {}
clean_dups = {}
new_names = []
for val in races_to_clean:
    new_name = column_rename_function(val)
    race_updates_dict[val] = new_name
    if new_name not in new_names:
        new_names.append(new_name)
        race_updates_reversed[new_name] = val
    else:
        print("Duplicate", new_name)
        print("\t",race_updates_reversed[new_name])
        print("\t",val)
        print("")
        clean_dups[val] = race_updates_reversed[new_name]

<>:107: SyntaxWarning: invalid escape sequence '\D'
<>:107: SyntaxWarning: invalid escape sequence '\D'
/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_26112/1298130022.py:107: SyntaxWarning: invalid escape sequence '\D'
  return re.sub("\D","",temp).zfill(fill_level)


In [13]:
race_updates_dict

{'YES-:-CA No. 1 (Act No. 408, 2024 - HB 300) -- Provides relative to federal proceeds from alternative energy production': 'G24A1YES',
 'NO-:-CA No. 1 (Act No. 408, 2024 - HB 300) -- Provides relative to federal proceeds from alternative energy production': 'G24A1NO',
 'Kamala D. Harris, Tim Walz Democratic (DEM)-:-Presidential Electors': 'G24PREDHAR',
 'Jill Stein, Ajamu Baraka Green (GRN)-:-Presidential Electors': 'G24PREOSTE',
 'Chase Oliver, Mike ter Maat Libertarian (LBT)-:-Presidential Electors': 'G24PRELOLI',
 'Donald J. Trump, "JD" Vance Republican (REP)-:-Presidential Electors': 'G24PRERTRU',
 'Peter Sonski, Lauren Onak American Solidarity Party (OTHER)-:-Presidential Electors': 'G24PREOSON',
 'Randall Terry, Stephen Broden Constitution Party (OTHER)-:-Presidential Electors': 'G24PREOTER',
 'Mattie Preston, Shannel Conner Godliness, Truth, Justice (OTHER)-:-Presidential Electors': 'G24PREOPRE',
 'Cornel West, Melina Abdullah Justice For All (OTHER)-:-Presidential Electors': '

In [14]:
# Export the Renamed Columns for the README
pd.DataFrame(race_updates_dict.items()).to_csv("./field_names_gen.csv", index = False)

# Rename the columns in the election results file
la_24_general.rename(columns = race_updates_dict, inplace = True)

In [15]:
la_24_comb = la_24_general.copy(deep = True)

In [16]:
fips_file = pd.read_csv("./raw-from-source/FIPS/US_FIPS_Codes.csv")
fips_file = fips_file[fips_file["State"]=="Louisiana"]
fips_file["FIPS County"] = fips_file["FIPS County"].astype(str)
fips_file["FIPS County"] = fips_file["FIPS County"].str.zfill(3)
fips_file["County Name"] = fips_file["County Name"].replace("La Salle","Lasalle")
fips_dict = dict(zip(fips_file['County Name'], fips_file['FIPS County']))
la_24_comb['COUNTYFP'] = la_24_comb['PARISH'].str.replace("St.","St").map(fips_dict).fillna(la_24_comb['PARISH'])
la_24_comb['COUNTYFP'] = la_24_comb['COUNTYFP'].astype(str).str.zfill(3)

# Print statements to check the county FIPs we've added
print(la_24_comb['COUNTYFP'].unique())

['001' '003' '005' '007' '009' '011' '013' '015' '017' '019' '021' '023'
 '025' '027' '029' '031' '033' '035' '037' '039' '041' '043' '045' '047'
 '049' '053' '051' '055' '057' '059' '061' '063' '065' '067' '069' '071'
 '073' '075' '077' '079' '081' '083' '085' '087' '089' '091' '093' '095'
 '097' '099' '101' '103' '105' '107' '109' '111' '113' '115' '117' '119'
 '121' '123' '125' '127']


In [17]:
# Sort the race columns in alphabetical order
race_cols = list(race_updates_dict.values())
race_cols.sort()

In [18]:
la_24_comb["UNIQUE_ID"] = la_24_comb["UNIQUE_ID"].str.strip()

In [19]:
# Print out the vote totals of all of the races for checking
for race in race_cols:
    la_24_comb[race] = la_24_comb[race].astype(int)
    print("~~"+race, sum(la_24_comb[race]))

~~G24A1NO 503275
~~G24A1YES 1367876
~~G24PREDHAR 766870
~~G24PRELOLI 6835
~~G24PREOCRU 1481
~~G24PREOFRU 361
~~G24PREOKEN 6641
~~G24PREOPRE 2857
~~G24PREOSON 2240
~~G24PREOSTE 7138
~~G24PREOTER 1424
~~G24PREOWES 2623
~~G24PRERTRU 1208505
~~GCON01DMAN 85911
~~GCON01NHYE 6781
~~GCON01RARR 17856
~~GCON01RSCA 238842
~~GCON01RSHA 8330
~~GCON02DCAR 184009
~~GCON02DDAV 32482
~~GCON02RGRA 39174
~~GCON02RLYN 41641
~~GCON02RPER 7878
~~GCON03DGON 59834
~~GCON03DSUM 21323
~~GCON03RHIG 226279
~~GCON03RJOH 13246
~~GCON04RJOH 262821
~~GCON04RMOR 43427
~~GCON05DVAL 82981
~~GCON05RLET 201037
~~GCON05RMEN 35833
~~GCON06DAND 23811
~~GCON06DFIE 150323
~~GCON06DJON 3910
~~GCON06DWIL 6252
~~GCON06RGUI 111737


In [20]:
la_24_comb = la_24_comb.fillna(0)

~~G24A1NO 503275~~
~~G24A1YES 1367876~~
~~G24PREDHAR 766870~~
~~G24PRELOLI 6835~~
~~G24PREOCRU 1481~~
~~G24PREOFRU 361~~
~~G24PREOKEN 6641~~
~~G24PREOPRE 2857~~
~~G24PREOSON 2240~~
~~G24PREOSTE 7138~~
~~G24PREOTER 1424~~
~~G24PREOWES 2623~~
~~G24PRERTRU 1208505~~
~~GCON01DMAN 85911~~
~~GCON01NHYE 6781~~
~~GCON01RARR 17856~~
~~GCON01RSCA 238842~~
~~GCON01RSHA 8330~~
~~GCON02DCAR 184009~~
~~GCON02DDAV 32482~~
~~GCON02RGRA 39174~~
~~GCON02RLYN 41641~~
~~GCON02RPER 7878~~
~~GCON03DGON 59834~~
~~GCON03DSUM 21323~~
~~GCON03RHIG 226279~~
~~GCON03RJOH 13246~~
~~GCON04RJOH 262821~~
~~GCON04RMOR 43427~~
~~GCON05DVAL 82981~~
~~GCON05RLET 201037~~
~~GCON05RMEN 35833~~
~~GCON06DAND 23811~~
~~GCON06DFIE 150323~~
~~GCON06DJON 3910~~
~~GCON06DWIL 6252~~
~~GCON06RGUI 111737~~

In [21]:
la_24_comb.rename(columns = {"PARISH":"Parish"}, inplace = True)
la_24_comb = la_24_comb[["UNIQUE_ID", "COUNTYFP", 'Parish', "Precinct"]+ race_cols]

In [22]:
la_24_comb["COUNTYFP"].unique()

array(['001', '003', '005', '007', '009', '011', '013', '015', '017',
       '019', '021', '023', '025', '027', '029', '031', '033', '035',
       '037', '039', '041', '043', '045', '047', '049', '053', '051',
       '055', '057', '059', '061', '063', '065', '067', '069', '071',
       '073', '075', '077', '079', '081', '083', '085', '087', '089',
       '091', '093', '095', '097', '099', '101', '103', '105', '107',
       '109', '111', '113', '115', '117', '119', '121', '123', '125',
       '127'], dtype=object)

In [23]:
if not os.path.exists("./la_2024_gen_prec_csv"):
    os.mkdir("./la_2024_gen_prec_csv")

In [ ]:
la_24_comb.to_csv("./la_2024_gen_prec_csv/la_2024_gen_prec.csv", index = False)

## To-Run After README Made

In [ ]:
import shutil

In [ ]:
shutil.make_archive("./la_2024_gen_prec_csv","zip","./la_2024_gen_prec_csv")